## EKS 환경 설정

아래 노트북은 원본 노트북에 설명을 추가 하였습니다.
- 실제 실행시에 여기의 셀의 "결과 값" 과 일치하는지를 확인하면서 실행 하십시오.
- 만일 에러등이 발생하면 확인하시고 다시 실행 해야 합니다.
    - https://github.com/data-science-on-aws/workshop/blob/master/12_kubeflow/00_01_Configure_Environment.ipynb

# Associate IAM Policies with EKS Worker Nodes


In [1]:
%%bash

### Source the environment
source ~/.bash_profile

export INSTANCE_ROLE_NAME=$(aws iam list-roles \
    | jq -r ".Roles[] \
    | select(.RoleName \
    | startswith(\"eksctl-$AWS_CLUSTER_NAME\") and contains(\"NodeInstanceRole\")) \
    .RoleName")
echo "export INSTANCE_ROLE_NAME=${INSTANCE_ROLE_NAME}" | tee -a ~/.bash_profile

export INSTANCE_PROFILE_ARN=$(aws iam list-roles \
    | jq -r ".Roles[] \
    | select(.RoleName \
    | startswith(\"eksctl-$AWS_CLUSTER_NAME\") and contains(\"NodeInstanceRole\")) \
    .Arn")
echo "export INSTANCE_PROFILE_ARN=${INSTANCE_PROFILE_ARN}" | tee -a ~/.bash_profile

export INSTANCE_ROLE_NAME=eksctl-cluster-nodegroup-cpu-node-NodeInstanceRole-19LK831GUGOC3
export INSTANCE_PROFILE_ARN=arn:aws:iam::079636235537:role/eksctl-cluster-nodegroup-cpu-node-NodeInstanceRole-19LK831GUGOC3


# Attach Roles

In [2]:
%%bash

### Source the environment
source ~/.bash_profile

#### Allow Access from/to the Elastic Container Registry (ECR)
# This allows our cluster worker nodes to load custom Docker images (ie. models) from ECR.  We will load these custom Docker images in a later section.
aws iam attach-role-policy --role-name $INSTANCE_ROLE_NAME --policy-arn arn:aws:iam::aws:policy/AmazonEC2ContainerRegistryFullAccess

echo "Completed"

Completed


# Associated IAM and OIDC
To use IAM Roles for Service Accounts in your cluster, you must create an OIDC identity provider in the IAM console.  

See https://docs.aws.amazon.com/eks/latest/userguide/enable-iam-roles-for-service-accounts.html for more info.

# _This may take a few minutes.  Please be patient._


In [3]:
%%bash

### Source the environment
source ~/.bash_profile

eksctl utils associate-iam-oidc-provider --cluster ${AWS_CLUSTER_NAME} --approve

echo "Completed"

2021-04-04 13:58:22 [ℹ]  eksctl version 0.43.0
2021-04-04 13:58:22 [ℹ]  using region us-east-1
2021-04-04 13:58:22 [ℹ]  will create IAM Open ID Connect provider for cluster "cluster" in "us-east-1"
2021-04-04 13:58:22 [✔]  created IAM Open ID Connect provider for cluster "cluster" in "us-east-1"
Completed


In [4]:
%%bash

### Source the environment
source ~/.bash_profile

aws eks describe-cluster --name ${AWS_CLUSTER_NAME} --region ${AWS_REGION} --query "cluster.identity.oidc.issuer" --output text

echo "Completed"

https://oidc.eks.us-east-1.amazonaws.com/id/71C77B45C6D0015ADA403E69FB9B8B17
Completed


# Update `~/.kube/config` with our new EKS cluster

In [5]:
%%bash

source ~/.bash_profile

aws eks --region ${AWS_REGION} update-kubeconfig --name ${AWS_CLUSTER_NAME} 

Added new context arn:aws:eks:us-east-1:079636235537:cluster/cluster to /home/ec2-user/.kube/config


# Verify Your EKS Cluster


In [6]:
%%bash

kubectl get ns


NAME              STATUS   AGE
default           Active   12m
kube-node-lease   Active   12m
kube-public       Active   12m
kube-system       Active   12m


In [7]:
%%bash

kubectl get nodes


NAME                             STATUS   ROLES    AGE    VERSION
ip-192-168-59-125.ec2.internal   Ready    <none>   4m5s   v1.18.9-eks-d1db3c
ip-192-168-6-128.ec2.internal    Ready    <none>   4m6s   v1.18.9-eks-d1db3c
